In [ ]:
import pandas as pd
from datasets import Dataset
import sys
import csv
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
import torch
!pip install rouge_score
!pip install evaluate
import evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a102d0226431c6a49eb7df121ead57dd4181355ee2b5f57cc2c55462e9eec483
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/PubMed Abstract Summarization Dataset/test.csv")  # Adjust path
test_df = test_df.dropna(subset=["article", "abstract"])
test_df = test_df[test_df['article'].str.strip() != '']
test_df = test_df[test_df['abstract'].str.strip() != '']

In [ ]:
model_path = "/content/drive/MyDrive/models/bart_pubmed_finetuned_latest1"
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
loaded_model.eval()

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [ ]:
def summarize(text, loaded_model, tokenizer, max_input_length=1024, max_output_length=1024):
    # Tokenize to tensors (includes attention_mask)
    enc = tokenizer(
        text,
        return_tensors="pt",
        max_length=max_input_length,
        truncation=True
    )

    # Send tensors to the same device as the model
    device = next(loaded_model.parameters()).device
    enc = {k: v.to(device) for k, v in enc.items()}

    # Generate (pass the full batch: input_ids + attention_mask)
    summary_ids = loaded_model.generate(
        **enc,
        max_length=max_output_length,
        min_length=30,
        num_beams=6,
        length_penalty=2.5,
        early_stopping=True,
        # Optional guardrails if repetition occurs:
        no_repeat_ngram_size=3,
        repetition_penalty=1.1,
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

rouge = evaluate.load("rouge")

# Test on a sample
sample_article = test_df['article'].iloc[0]
print("Original:\n", sample_article[:1000])
predicted_summary = summarize(sample_article, loaded_model, tokenizer)
print("\nPredicted Summary:\n", predicted_summary)
results = rouge.compute(predictions=[predicted_summary], references=[test_df['abstract'].iloc[0]])
print(results)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Original:
 anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] . 
 although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet , 
 our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression . 
 overall , neuropsychiatric symptoms in pd have been shown to be negatively associated with cognitive performance . 
 for example , higher depression scores have been correlated with lower scores on the mini - mental state exam ( mmse ) [ 8 , 9 ] as well as tests of memory and executive functions ( e.g. , attention ) [ 1014 ] . 
 likewise , apathy and anhedonia in pd patients have been associated with execut

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1737: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



Predicted Summary:
 anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression . 
 our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression . given that research on healthy young adults suggests that anxiety reduces processing capacity and impairs processing efficiency , especially in the central executive and attentional systems of working memory , we hypothesized that pd patients with anxiety would show impairments in attentional set - shifting and working memory compared to patients without anxiety . to test this hypothesis 
 , we examined the independent effect of anxiety on attention and memory performance in a sample of pd ( n = 17 ) patients with and without anxiety using the mini - mental state exam ( mmse ) , the timed up and go test ( tug ) , and 

In [ ]:
preds = []
refs = []
for i in range(100):  # evaluate on first 100 samples
    input_text = test_df['article'].iloc[i]
    reference = test_df['abstract'].iloc[i]
    pred = summarize(input_text, loaded_model, tokenizer)
    preds.append(pred)
    refs.append(reference)

results = rouge.compute(predictions=preds, references=refs)
print(results)

{'rouge1': np.float64(0.4388349833459976), 'rouge2': np.float64(0.1792071230303051), 'rougeL': np.float64(0.2555997204231656), 'rougeLsum': np.float64(0.3850250018140928)}
